**<h1 align="center">CSV Preprocessing</h1>**

## Global Variables

### Project Specific Variables

### Colors

In [ ]:
# ANSI escape codes for colored output
ANSI = {
    'R' : '\033[91m',  # Red
    'G' : '\033[92m',  # Green
    'B' : '\033[94m',  # Blue
    'Y' : '\033[93m',  # Yellow
    'W' : '\033[0m',  # White
}

## Import CSV to Dataframe

### CSV Import